In [ ]:
!pip install -q langchain huggingface_hub transformers sentence_transformers 
!pip install -q accelerate bitsandbytes safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:00

In [ ]:
!git lfs install

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [ ]:
!nvidia-smi

Thu May 11 05:16:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git lfs install
!git clone https://huggingface.co/TheBloke/wizardLM-7B-GPTQ ./models/wizardLM-7B-GPTQ

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into './models/wizardLM-7B-GPTQ'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 100 (delta 51), reused 2 (delta 2), pack-reused 0
Receiving objects: 100% (100/100), 492.56 KiB | 10.71 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Filtering content: 100% (3/3), 7.25 GiB | 52.54 MiB/s, done.


In [ ]:
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
# !cp ./GPTQ-for-LLaMa/setup_cuda.py ./GPTQ-for-LLaMa/setup.py
# !cd GPTQ-for-LLaMa && python setup_cuda.py install
!cd GPTQ-for-LLaMa && python3 setup_cuda.py bdist_wheel -d .

In [ ]:
!pip install ./GPTQ-for-LLaMa/*.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./GPTQ-for-LLaMa/quant_cuda-0.0.0-cp310-cp310-linux_x86_64.whl


> **Note:** If you get `NameError: No module named quant_cuda`, restart the kernel and reinstall GPTQ in the above cell

In [ ]:
from pathlib import Path

import sys
sys.path.insert(0, str(Path("/content/GPTQ-for-LLaMa")))

import os
import glob
import site 
import torch
import logging
import accelerate
import inspect

import numpy as np
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    LlamaTokenizer,
    AutoConfig, 
    AutoModelForCausalLM
)

In [ ]:
try:
    from modelutils import find_layers
except ImportError:
    from utils import find_layers


try:
    from quant import make_quant
    is_triton = False
except ImportError:
    import quant
    is_triton = True

In [ ]:
GPTQ_MODEL_DIR = "/content/models/"
MODEL_NAME = "wizardLM-7B-GPTQ"

## Helpers

In [ ]:
# Based on https://github.com/oobabooga/text-generation-webui/blob/main/modules/GPTQ_loader.py
def find_quantized_model_file(model_name, args):
    if args.checkpoint:
        return Path(args.checkpoint)

    path_to_model = Path(f'{args.model_dir}/{model_name}')
    print(f"Path to Model: {path_to_model}")
    pt_path = None
    priority_name_list = [
        Path(f'{args.model_dir}/{model_name}{hyphen}{args.wbits}bit{group}{ext}')
        for group in ([f'-{args.groupsize}g', ''] if args.groupsize > 0 else [''])
        for ext in ['.safetensors', '.pt']
        for hyphen in ['-', f'/{model_name}-', '/']
    ]
    for path in priority_name_list:
        if path.exists():
            pt_path = path
            break

    # If the model hasn't been found with a well-behaved name, pick the last .pt
    # or the last .safetensors found in its folder as a last resort
    if not pt_path:
        found_pts = list(path_to_model.glob("*.pt"))
        found_safetensors = list(path_to_model.glob("*.safetensors"))
        pt_path = None

        if len(found_pts) > 0:
            if len(found_pts) > 1:
                logging.warning('More than one .pt model has been found. The last one will be selected. It could be wrong.')

            pt_path = found_pts[-1]
        elif len(found_safetensors) > 0:
            if len(found_pts) > 1:
                logging.warning('More than one .safetensors model has been found. The last one will be selected. It could be wrong.')

            pt_path = found_safetensors[-1]

    return pt_path

In [ ]:
# Section Inspired from https://github.com/oobabooga/text-generation-webui
def _load_quant(model, checkpoint, wbits, groupsize=-1, faster_kernel=False, eval=False, exclude_layers=['lm_head'], kernel_switch_threshold=128):
    def noop(*args, **kwargs):
        pass

    config = AutoConfig.from_pretrained(model, trust_remote_code=True)
    logging.info(f"Model Config: {config}")
    
    torch.nn.init.kaiming_uniform_ = noop
    torch.nn.init.uniform_ = noop
    torch.nn.init.normal_ = noop

    torch.set_default_dtype(torch.half)
    transformers.modeling_utils._init_weights = False
    torch.set_default_dtype(torch.half)
    model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    torch.set_default_dtype(torch.float)
    
    if eval:
        model = model.eval()

    layers = find_layers(model)
    for name in exclude_layers:
        if name in layers:
            del layers[name]

    if not is_triton:
        gptq_args = inspect.getfullargspec(make_quant).args
        make_quant_kwargs = {
                'module': model,
                'names': layers,
                'bits': wbits,
            }
        if 'groupsize' in gptq_args:
            make_quant_kwargs['groupsize'] = groupsize
        if 'faster' in gptq_args:
            make_quant_kwargs['faster'] = faster_kernel
        if 'kernel_switch_threshold' in gptq_args:
            make_quant_kwargs['kernel_switch_threshold'] = kernel_switch_threshold
        
        make_quant(**make_quant_kwargs)
    else:
        logging.exception("Triton not supported!")

    del layers

    if checkpoint.endswith('.safetensors'):
        from safetensors.torch import load_file as safe_load
        model.load_state_dict(safe_load(checkpoint), strict=False)
    else:
        model.load_state_dict(torch.load(checkpoint), strict=False)

    model.seqlen = 2048
    return model


In [ ]:
# Section Inspired from https://github.com/oobabooga/text-generation-webui
def load_quantized_model(model_name, args, load_tokenizer=True):
    tokenizer = None
    path_to_model = Path(f'{args.model_dir}/{model_name}')
    pt_path = find_quantized_model_file(model_name, args)
    if not pt_path:
        print(pt_path)
        logging.error("Could not find the quantized model in .pt or .safetensors format, exiting...")
        return
    else:
        logging.info(f"Found the following quantized model: {pt_path}")
    
    threshold = args.threshold if args.threshold else 128

    model = _load_quant(
        str(path_to_model), 
        str(pt_path), 
        args.wbits, 
        args.groupsize, 
        kernel_switch_threshold=threshold
    )

    model = model.to(torch.device("cuda:0"))

    if load_tokenizer:
        tokenizer = LlamaTokenizer.from_pretrained(
            Path(f"{args.model_dir}/{model_name}/"), 
            clean_up_tokenization_spaces=True
        )

        try:
            tokenizer.eos_token_id = 2
            tokenizer.bos_token_id = 1
            tokenizer.pad_token_id = 0
        except:
            pass

    return model, tokenizer

In [ ]:
class AttributeDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

## WizardLM + Langchain

In [ ]:
args = {
    "wbits": 4,
    "groupsize": 128,
    "model_type": "llama",
    "model_dir": GPTQ_MODEL_DIR,
}

model, tokenizer = load_quantized_model(MODEL_NAME, args=AttributeDict(args))

Path to Model: /content/models/wizardLM-7B-GPTQ


In [ ]:
max_memory = {
    0: "15360MiB",
    'cpu': "12GiB"
}

In [ ]:
device_map = accelerate.infer_auto_device_map(
    model, 
    max_memory=max_memory, 
    no_split_module_classes=["LlamaDecoderLayer"]
)
model = accelerate.dispatch_model(
    model, 
    device_map=device_map, 
    offload_buffers=True
)

In [ ]:
model.get_memory_footprint() / (1024 * 1024)

3874.857421875

In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    device_map=device_map
)

local_llm = HuggingFacePipeline(pipeline=llm_pipeline)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt, 
    llm=local_llm
)

In [ ]:
print(llm_chain.run('Who invented the light bulb?'))

 The light bulb is a device that produces light by using electricity. It was invented by many people over the years, but the most famous inventor of the light bulb is Thomas Edison. He invented the first practical and commercially successful light bulb in 1879.
